In [1]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import time
import re
import pickle

In [2]:
query_txt = '대전 지역경제' # 검색어
start_date = '20180101'
end_date = '20201231'

In [3]:
file_path = 'C:\대전 크롤링_추가15분야\\' # 파일 경로 바꿀것
path = file_path + 'chromedriver.exe' #크롬드라이버
driver = webdriver.Chrome(path)

C:\Users\cho03\AppData\Local\Temp\ipykernel_3964\1370352008.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)
Incompatible release of chromedriver (version 113.0.5672.63) detected in PATH: C:\Users\cho03\Downloads\���̳�\ũ�Ѹ� ����\code\Crawling\chromedriver.exe


In [4]:
driver.get('https://www.naver.com')
time.sleep(3) #페이지 켜지는 시간 고려

element = driver.find_element(By.ID, 'query')
element.send_keys(query_txt) #검색어 입력
element.submit() #엔터

driver.find_element_by_link_text("블로그").click()
time.sleep(3)

AttributeError: 'WebDriver' object has no attribute 'find_element_by_id'

In [ ]:
driver.find_element_by_id("search_option_button").click()
time.sleep(6)
print("ok")
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/a""").click() #정렬
time.sleep(3)
print(1)

In [ ]:
driver.find_element_by_xpath("""//*[@id='snb']/div/ul/li[1]/div/ul/li[1]/a""").click() #관련도
time.sleep(3)
print(1)
driver.find_element_by_xpath("//*[@id='snb']/div/ul/li[2]/a").click() #기간
time.sleep(3)
print("ok")

In [ ]:
s_date = driver.find_element_by_xpath('//*[@id="date_from_input"]') #지정 start date로 입력
time.sleep(3)
s_date.click()
time.sleep(5)
s_date.send_keys(start_date)

In [ ]:
e_date = driver.find_element_by_xpath('//*[@id="date_to_input"]')
time.sleep(3)
e_date.click()
time.sleep(5)
e_date.send_keys(end_date)

In [ ]:
time.sleep(3)
driver.find_element_by_class_name("tx").click() #적용

In [ ]:
import datetime
    
def doScrollDown(whileSeconds):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=whileSeconds)
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
        if datetime.datetime.now() > end:
            break

In [ ]:
doScrollDown(100) #스크롤 지속시간

In [ ]:
from tqdm import tqdm
title_list = []
doc_list=[]
url_list = []

for i in tqdm(range(0,10)):
    try:
        for_url = driver.find_element_by_xpath('//*[@id="sp_blog_{}"]/div/div[2]/div/a'.format(i+1))
    except:
        for_url = driver.find_element_by_xpath('//*[@id="sp_blog_{}"]/div/a'.format(i+1)) #문서 사진 없는 경우 오류 방지
    doc = for_url.text
    url = for_url.get_attribute('href')

    try:
        title = driver.find_element_by_xpath('//*[@id="sp_blog_{}"]/div/a[2]'.format(i+1)).text
    except:
        title = driver.find_element_by_xpath('//*[@id="sp_blog_{}"]/div/div[2]/div/a/div'.format(i+1)).text

    title_list.append(title)
    url_list.append(url)
    doc_list.append(doc)

In [ ]:
len(title_list)   ###

In [ ]:
raw_data = pd.DataFrame()
raw_data['title'] = title_list
raw_data['doc'] = doc_list
raw_data['url'] = url_list

In [ ]:
#raw_data

In [ ]:
#내용 크롤링

In [ ]:
new_doc = []
for i in range(0,len(url_list)):
    url_path = url_list[i]
    driver.switch_to_window(driver.window_handles[0]) # 탭 0번 전환(켜진 순서)
    driver.execute_script("window.open('{}')".format(url_path)) #문서 순서대로 탭 켜짐
    driver.switch_to_window(driver.window_handles[1]) # 탭 1번 전환(켜진 탭으로)
    time.sleep(5)
    iframes = driver.find_elements_by_tag_name('iframe')
    if len(iframes) > 0: #iframe 있는 경우만
        try:
            driver.switch_to_frame(0) # 첫번째 iframe
            html = driver.page_source #개발자도구
            soup = BS(html, 'html.parser') #개발자도구
            try:
                a = soup.find('div', class_='se-main-container').get_text() #iframe 속 container에서 text만 
            except :
                a = soup.find('div', id='postListBody') #get.text() 못씀
                a = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", str(a)) #한글만
            new_doc.append(a)
            driver.switch_to_default_content() #프레임 전환하기
        except: #
            print(i)
    else:
        print(i)
    driver.close() #켜진 탭 하나(긁어온 탭)만 종료   #driver.quit() : 켜진 크롬 다 종료
    time.sleep(5)
    if i%10 == 0:
        print(i)

In [ ]:
len(new_doc)

In [ ]:
raw_data = pd.DataFrame()
raw_data['title'] = title_list
raw_data['doc'] = new_doc
raw_data['url'] = url_list
raw_data['ch'] = 'naver'
raw_data['ch2'] = 'blog'

In [ ]:
#raw_data

In [ ]:
driver.quit() #종료

In [ ]:
f = open(file_path + "\\1_naver blog.pkl", "wb")
pickle.dump(raw_data, f)

f.close()